In [139]:
import nltk
import util
import json
import frequencies as fr
import topic_modeling as tm
import content_class as content
import ngram_analysis as sent
import pandas as pd
%reload_ext autoreload
%autoreload 2

In [2]:
beach = ['zihuatanejo', 'sayulita', 'puerto escondido', 'todos santos',
       'huatulco', 'zihuatanejo de azueta', 'cabo san lucas', 'rosarito',
       'punta de mita', 'manzanillo']

nature = ['valle de bravo', 'comitan', 'ciudad valles', 'san carlos', 'colima',
       'palenque', 'monterrey', 'tepoztlan', 'tuxtla gutierrez', 'oaxaca']

culture = ['xalapa', 'cuernavaca', 'mexico city', 'san cristobal de las casas',
       'saltillo', 'chihuahua', 'oaxaca', 'ciudad juarez', 'aguascalientes',
       'monterrey']

shopping = ['zapopan', 'leon', 'ciudad juarez', 'tijuana', 'hermosillo', 'cancun',
       'patzcuaro', 'tlaquepaque', 'chihuahua', 'puerto penasco']

In [3]:
segments_clean = util.open_json('data\segments_cleaned.json')

In [106]:
beach_segclean = util.get_city_segments(segments_clean, beach)
cluture_segclean = util.get_city_segments(segments_clean, culture)
nature_segclean = util.get_city_segments(segments_clean, nature)
shopping_segclean = util.get_city_segments(segments_clean, shopping)

In [108]:
cluture_segclean['cuernavaca']['segments']

[" I have heard to other reviewers here that the place is under renovation, so apparently people can't go in right now. If it ever opens, I really recommend checking it out.",
 " I have heard to other reviewers here that the place is under renovation, so apparently people can't go in right now. However, when I went in 2010, it was open to the public and absolutely amazing. If it ever opens, I really recommend checking it out.",
 'As of March 2019 the palace is undergoing renovations and only two small rooms are open.',
 " Small temporary exhibit of archaeological finds only. Can't really appreciate the exterior of the palace either as lots of scaffolding.",
 " At time of writing (February 2019) it's about the artistic treatment of animals by the indigenous populations from earliest days. It's quite small but well displayed and free.",
 ' Built in in 1526 it is considered as the oldest colonial civil structure in the whole Americas. It is a Gothic Mudejar fortified building which remind

# Getting segments topics

In [5]:
cities_corpus = util.open_json('data\cities_corpus.json')

In [6]:
all_cities = {k:v for val in cities_corpus.values() for k,v in val.items()}

In [7]:
cities = util.open_json('data/cities.json')
states = util.open_json('data/states.json')
states_corpus = {k: v['clean'] for k, v in states.items()}
cities_ = [t for v in cities.values() for t in nltk.word_tokenize(v['name'])]
states_ = [t for v in list(states_corpus.keys()) for t in nltk.word_tokenize(v)]
places = list(set(cities_+states_))

In [8]:
all_cities_bis = {}
for k, v in all_cities.items():
    tokens = v.split()
    valid_words=[t for t in tokens if t not in places]
    all_cities_bis[k] = ' '.join(valid_words)

In [9]:
cities_topics_bis = tm.Topics(all_cities_bis, 20)

In [10]:
cities_topics_bis.z_matrix('lda')
cities_topics_bis.z_matrix('nmf')
cities_topics_bis.z_matrix('lsi')
print()

c:\users\jesus\appdata\local\programs\python\python37\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [11]:
for i,r in cities_topics_bis.topic_words('nmf').iterrows():
    print(i, '|', r['Content'])
    print('**********************************************************')

0 | beach, waves, sand, surf, beaches, bay, fish, ocean, snorkeling, swimming
**********************************************************
1 | que, es, en, para, muy, con, lugar, una, se, lo
**********************************************************
2 | lake, lagoon, fort, boat, swim, cenotes, laguna, pirates, lakes, swimming
**********************************************************
3 | dolphins, dolphin, swim, beach, photos, trainer, package, manatees, ferry, sea
**********************************************************
4 | ruins, mayan, site, dzibanche, guide, tour, jungle, maya, museum, cruise
**********************************************************
5 | canyon, hike, trail, hiking, zip, volcano, views, boat, climb, estuary
**********************************************************
6 | pyramid, church, climb, tunnels, hike, site, views, convent, temple, hill
**********************************************************
7 | cenote, cenotes, tour, guide, swim, cave, mayan, fish, clear, 

In [12]:
cont = content.Analysis(cities_topics_bis, 'nmf')

In [13]:
cont.set_importan_topics()

Enter the indeces of the topics you consider important, separated by commas:0,5,9,10


In [144]:
beach_to_process = util.segments_to_process(cont, segments_clean, beach, 0, 0.2)
nature_to_process = util.segments_to_process(cont, segments_clean, nature, 5, 0.1)
culture_to_process = util.segments_to_process(cont, segments_clean, culture, 9, 0.2)
shopping_to_process = util.segments_to_process(cont, segments_clean, shopping, 10, 0.1)

In [151]:
# with open('data/shopping_to_process.json', 'w') as outfile:
#         json.dump(shopping_to_process, outfile)

In [146]:
sum([len(v) for v in beach_to_process.values()])

2182

# Sentiment Analysis

In [127]:
nature_sent_val = sent.reviews_sentiment(nature_to_process)

In [136]:
nature_sent_val_b ={k:v for k, v in nature_sent_val.items() if len(v) > 10}

In [137]:
nature_sent_ratios = sent.sentiment_ratios(nature_sent_val_b)

In [143]:
pd.DataFrame.from_dict(nature_sent_ratios, orient='index').sort_values(by='positive', ascending=False)

,negative,neutral,positive
valle de bravo,0.33,0.25,0.42
oaxaca,0.45,0.22,0.33
palenque,0.41,0.29,0.29
monterrey,0.33,0.42,0.25
ciudad valles,0.42,0.36,0.22
comitan,0.42,0.36,0.22


In [152]:
shopping_sent_val = sent.reviews_sentiment(shopping_to_process)

In [154]:
shopping_sent_val_b ={k:v for k, v in shopping_sent_val.items() if len(v) > 10}

In [155]:
shopping_sent_ratios = sent.sentiment_ratios(shopping_sent_val_b)

In [157]:
pd.DataFrame.from_dict(shopping_sent_ratios, orient='index').sort_values(by='positive', ascending=False)

,negative,neutral,positive
zapopan,0.28,0.30,0.42
cancun,0.34,0.31,0.35
ciudad juarez,0.28,0.39,0.33
puerto penasco,0.31,0.38,0.31
hermosillo,0.55,0.18,0.27
tijuana,0.39,0.34,0.27
leon,0.50,0.27,0.23
chihuahua,0.69,0.15,0.15


In [158]:
culture_sent_val = sent.reviews_sentiment(culture_to_process)

In [159]:
culture_sent_val_b ={k:v for k, v in culture_sent_val.items() if len(v) > 10}

In [160]:
culture_sent_ratios = sent.sentiment_ratios(culture_sent_val_b)

In [161]:
pd.DataFrame.from_dict(culture_sent_ratios, orient='index').sort_values(by='positive', ascending=False)

,negative,neutral,positive
cuernavaca,0.30,0.29,0.42
aguascalientes,0.35,0.26,0.39
oaxaca,0.34,0.28,0.37
san cristobal de las casas,0.39,0.24,0.37
ciudad juarez,0.28,0.36,0.36
monterrey,0.40,0.26,0.34
mexico city,0.32,0.35,0.32
chihuahua,0.42,0.32,0.26
saltillo,0.39,0.39,0.22
xalapa,0.49,0.29,0.22


In [162]:
beach_sent_val = sent.reviews_sentiment(beach_to_process)

In [163]:
beach_sent_val_b ={k:v for k, v in beach_sent_val.items() if len(v) > 10}

In [164]:
beach_sent_ratios = sent.sentiment_ratios(beach_sent_val_b)

In [165]:
pd.DataFrame.from_dict(beach_sent_ratios, orient='index').sort_values(by='positive', ascending=False)

,negative,neutral,positive
punta de mita,0.00,0.33,0.67
puerto escondido,0.13,0.24,0.63
zihuatanejo,0.15,0.27,0.58
zihuatanejo de azueta,0.14,0.32,0.54
cabo san lucas,0.20,0.30,0.50
rosarito,0.16,0.37,0.48
huatulco,0.19,0.34,0.47
todos santos,0.19,0.37,0.45
manzanillo,0.13,0.42,0.44
sayulita,0.22,0.34,0.44


In [166]:
beach_sent_ratios

{'zihuatanejo': {'negative': 0.15, 'neutral': 0.27, 'positive': 0.58},
 'sayulita': {'negative': 0.22, 'neutral': 0.34, 'positive': 0.44},
 'puerto escondido': {'negative': 0.13, 'neutral': 0.24, 'positive': 0.63},
 'todos santos': {'negative': 0.19, 'neutral': 0.37, 'positive': 0.45},
 'huatulco': {'negative': 0.19, 'neutral': 0.34, 'positive': 0.47},
 'zihuatanejo de azueta': {'negative': 0.14,
  'neutral': 0.32,
  'positive': 0.54},
 'cabo san lucas': {'negative': 0.2, 'neutral': 0.3, 'positive': 0.5},
 'rosarito': {'negative': 0.16, 'neutral': 0.37, 'positive': 0.48},
 'punta de mita': {'negative': 0.0, 'neutral': 0.33, 'positive': 0.67},
 'manzanillo': {'negative': 0.13, 'neutral': 0.42, 'positive': 0.44}}